# hitmen - Extended Feature Selection
* StelllarAlgo Data Science
* Ryan Kazmerik & Nakisa Rad
* Feb 25, 2022

## Hypothesis
Two very important components of a machine learning model are feature selection and feature engineering. Our idea is that adding some more features to the StellarAlgo retention model could improve performance of the model. This notebook will test the standard StellarAlgo retention model features.

## Experiment
This section details our experiment including querying data, data transformations, feature selection and modelling.

In [1]:
import getpass
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt

from pycaret.classification import *

warnings.filterwarnings('ignore')

### Let's connect to MSSQL and run a stored proc to get our dataset:

In [2]:
# connect to SQL Server.
SERVER = '34.206.73.189'  
DATABASE = 'datascience' 
USERNAME = 'nrad' 
PASSWORD = getpass.getpass(prompt='Enter your password')
CNXN = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+SERVER+';DATABASE='+DATABASE+';UID='+USERNAME+';PWD='+ PASSWORD)

In [3]:
lkupclientid = 37 # hitmen

cursor = CNXN.cursor()

storedProc = (
    f"""Exec [stlrWHL].[ds].[getRetentionScoringModelData] {lkupclientid}"""
)

df = pd.read_sql(storedProc, CNXN)

# apply some data transformations
df["year"] = pd.to_numeric(df["year"])

CNXN.commit()
cursor.close()

df.head()
#df.info()

,lkupClientId,dimCustomerMasterId,customerNumber,year,productGrouping,totalSpent,recentDate,attendancePercent,renewedBeforeDays,isBuyer,source_tenure,tenure,distToVenue,totalGames,recency,missed_games_1,missed_games_2,missed_games_over_2,click_link,fill_out_form,open_email,send_email,unsubscribe_email,openToSendRatio,clickToSendRatio,clickToOpenRatio,posting_records,resale_records,resale_atp,forward_records,cancel_records,email,inbound_email,inbound_phonecall,inperson_contact,internal_note,left_message,outbound_email,outbound_phonecall,phonecall,text,unknown,gender,childrenPresentInHH,maritalStatus,lengthOfResidenceInYrs,annualHHIncome,education,urbanicity,credits_after_refund,is_Lockdown,NumberofGamesPerSeason,CNTPostponedGames,isNextYear_Buyer
0,37,5206,9832738,2015,Flex Plan,100.0,2016-01-03,1.0,11,TRUE,2920,65,181.73,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,36,None,0
1,37,15396,10839176,2015,Flex Plan,40.0,2015-10-29,1.0,8,TRUE,2555,19,13.23,2,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,36,None,0
2,37,19570,11043392,2015,Flex Plan,37.5,2015-12-30,1.0,7,TRUE,2555,7,17.15,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,36,None,0
3,37,25958,9936091,2015,Flex Plan,50.0,2016-01-03,1.0,17,TRUE,2920,17,17.15,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,36,None,0
4,37,42288,10895932,2015,Flex Plan,37.5,2015-12-28,1.0,6,TRUE,2555,6,16.35,1,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,Unknown,1,1,None,None,None,None,0.0,0,36,None,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5713 entries, 0 to 5712
Data columns (total 54 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   lkupClientId            5713 non-null   int64  
 1   dimCustomerMasterId     5713 non-null   int64  
 2   customerNumber          5713 non-null   object 
 3   year                    5713 non-null   int64  
 4   productGrouping         5713 non-null   object 
 5   totalSpent              5713 non-null   float64
 6   recentDate              5713 non-null   object 
 7   attendancePercent       5713 non-null   float64
 8   renewedBeforeDays       5713 non-null   int64  
 9   isBuyer                 5713 non-null   object 
 10  source_tenure           5713 non-null   int64  
 11  tenure                  5713 non-null   int64  
 12  distToVenue             5713 non-null   float64
 13  totalGames              5713 non-null   int64  
 14  recency                 5713 non-null   

### Let's also drop the features that only have a single value, as they won't add much differentiation to our model:

In [5]:
for col in df.columns:
    if len(df[col].unique()) == 1:
        df.drop(col,inplace=True,axis=1)
        
df.shape

(5713, 29)

### We should also drop features that have a low correlation with the target label as they won't be useful for prediction, we'll only keep features that have a correlation above a set threshold:

In [6]:
cor = df.corr()

threshold = 0.05

#Correlation with output variable
cor_target = abs(cor["isNextYear_Buyer"])

#Selecting highly correlated features
relevant_features = cor_target[cor_target > threshold]

feats = []
for name, val in relevant_features.items():
    feats.append(name)

df_correlated = df[feats]

df_correlated.shape

df_correlated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5713 entries, 0 to 5712
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   year                    5713 non-null   int64
 1   renewedBeforeDays       5713 non-null   int64
 2   source_tenure           5713 non-null   int64
 3   tenure                  5713 non-null   int64
 4   totalGames              5713 non-null   int64
 5   missed_games_1          5713 non-null   int64
 6   missed_games_2          5713 non-null   int64
 7   missed_games_over_2     5713 non-null   int64
 8   is_Lockdown             5713 non-null   int64
 9   NumberofGamesPerSeason  5713 non-null   int64
 10  isNextYear_Buyer        5713 non-null   int64
dtypes: int64(11)
memory usage: 491.1 KB


### Now that we have the right features we can look at the correlations between them, if features are highly correlated with each other it might negatively impact the model:

In [7]:
corr = df_correlated.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

,year,renewedBeforeDays,source_tenure,tenure,totalGames,missed_games_1,missed_games_2,missed_games_over_2,is_Lockdown,NumberofGamesPerSeason,isNextYear_Buyer
year,1.00,-0.58,0.05,0.80,-0.16,-0.16,-0.08,-0.04,0.75,-0.77,-0.54
renewedBeforeDays,-0.58,1.00,0.20,-0.28,0.38,0.29,0.21,0.27,-0.48,0.51,0.62
source_tenure,0.05,0.20,1.00,0.32,-0.01,0.03,0.06,0.26,0.02,-0.01,0.16
tenure,0.80,-0.28,0.32,1.00,0.00,-0.05,0.01,0.08,0.61,-0.58,-0.33
totalGames,-0.16,0.38,-0.01,0.00,1.00,0.56,0.33,-0.05,-0.22,0.18,0.30
missed_games_1,-0.16,0.29,0.03,-0.05,0.56,1.00,0.34,-0.08,-0.19,0.18,0.21
missed_games_2,-0.08,0.21,0.06,0.01,0.33,0.34,1.00,0.05,-0.10,0.12,0.14
missed_games_over_2,-0.04,0.27,0.26,0.08,-0.05,-0.08,0.05,1.00,-0.09,0.06,0.22
is_Lockdown,0.75,-0.48,0.02,0.61,-0.22,-0.19,-0.10,-0.09,1.00,-0.50,-0.42
NumberofGamesPerSeason,-0.77,0.51,-0.01,-0.58,0.18,0.18,0.12,0.06,-0.50,1.00,0.28


### In order to compare two sets of features, we need to create some datasets for training and evalution:


In [8]:
# select % of the data for training
df_train = df_correlated.sample(frac=0.8, random_state=786).reset_index(drop=True)

# create the eval datasets for A and B
df_eval = df_correlated.drop(df_train.index).reset_index(drop=True)

# print out the number of records for training and eval
print('Data for Modeling: ' + str(df_train.shape))
print('Unseen Data For Predictions: ' + str(df_eval.shape), end="\n\n")

Data for Modeling: (4570, 11)
Unseen Data For Predictions: (1143, 11)



### Now we can model the data using a binary classification prediction for the isnextyear_buyer field to see how likely a customer is to re-purchase.

In [9]:
setup(
    data= df_train, 
    target="isNextYear_Buyer", 
    train_size = 0.80,
    data_split_shuffle=True,
    silent=True,
    numeric_features=[
        "renewedBeforeDays",
        "tenure",
        "source_tenure",
        "totalGames",
        "missed_games_1",
        "missed_games_2",
        "missed_games_over_2",
        "is_Lockdown",
        "NumberofGamesPerSeason"
    ]
)

,Description,Value
0,session_id,2028
1,Target,isNextYear_Buyer
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(4570, 11)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,1
8,Ordinal Features,False
9,High Cardinality Features,False


(False,
 False,
 {'lr': <pycaret.containers.models.classification.LogisticRegressionClassifierContainer at 0x7f92832e8550>,
  'knn': <pycaret.containers.models.classification.KNeighborsClassifierContainer at 0x7f92832e8ee0>,
  'nb': <pycaret.containers.models.classification.GaussianNBClassifierContainer at 0x7f92832e89d0>,
  'dt': <pycaret.containers.models.classification.DecisionTreeClassifierContainer at 0x7f92832e8b80>,
  'svm': <pycaret.containers.models.classification.SGDClassifierContainer at 0x7f928357a5b0>,
  'rbfsvm': <pycaret.containers.models.classification.SVCClassifierContainer at 0x7f92833099d0>,
  'gpc': <pycaret.containers.models.classification.GaussianProcessClassifierContainer at 0x7f9283309af0>,
  'mlp': <pycaret.containers.models.classification.MLPClassifierContainer at 0x7f9283309bb0>,
  'ridge': <pycaret.containers.models.classification.RidgeClassifierContainer at 0x7f9283571190>,
  'rf': <pycaret.containers.models.classification.RandomForestClassifierContainer at

In [10]:
model_matrix = compare_models(
    fold=10,
    include=["lr", "xgboost"]
)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.8972,0.9331,0.9491,0.8679,0.9066,0.7927,0.7968,0.3290
lr,Logistic Regression,0.8920,0.9341,0.9511,0.8592,0.9026,0.7820,0.7876,0.4190


In [11]:
best_model = create_model(model_matrix)
final_model = finalize_model(best_model)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8852,0.9249,0.9223,0.8683,0.8945,0.7690,0.7707
1,0.8743,0.8998,0.9430,0.8387,0.8878,0.7461,0.7527
2,0.8989,0.9311,0.9482,0.8714,0.9082,0.7962,0.7997
3,0.9098,0.9391,0.9479,0.8878,0.9169,0.8186,0.8207
4,0.9016,0.9447,0.9479,0.8750,0.9100,0.8019,0.8051
5,0.8934,0.9326,0.9531,0.8592,0.9037,0.7852,0.7905
6,0.8849,0.9340,0.9323,0.8606,0.8950,0.7682,0.7712
7,0.9178,0.9341,0.9844,0.8750,0.9265,0.8340,0.8415
8,0.9041,0.9482,0.9583,0.8720,0.9132,0.8066,0.8111
9,0.9014,0.9426,0.9531,0.8714,0.9104,0.8012,0.8052


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Library/Frameworks/Py

## Observations
Here you can document some ideas on the results from above

## Conclusions
Here you can talk about next steps, did the experiment work? If yes, what to do next? If no, why?